In [2]:
import numpy as np
import pandas as pd
import pickle
%matplotlib inline
import matplotlib # 注意这个也要import一次
import matplotlib.pyplot as plt
import seaborn as sns

import gc

from datetime import date


from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, mean_squared_error, roc_curve, auc
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [3]:
# 每日天气
weather_daily = pd.read_csv(r"/home/aistudio/data/data141580/每日天气数据.csv", encoding='utf-16')
# 每小时天气
weather_hour = pd.read_csv(r"/home/aistudio/data/data141580/每小时天气数据.csv", encoding='utf-16')

In [4]:
# 室内温度
inner_temp_hour = pd.read_csv(r"/home/aistudio/data/data141580/每小时室内温度监测数据.csv", encoding='utf-16')
# 室内湿度
inner_humidity_hour = pd.read_csv(r"/home/aistudio/data/data141580/每小时室内湿度监测数据.csv", encoding='utf-16')
# PM2.5
inner_PM25_hour = pd.read_csv(r"/home/aistudio/data/data141580/每小时室内PM2.5监测数据.csv", encoding='utf-16')
# CO2
inner_CO2_hour = pd.read_csv(r"/home/aistudio/data/data141580/每小时室内CO2监测数据.csv", encoding='utf-16')

### 1. 处理室内环境数据

In [5]:
# inner_temp_hour

In [6]:
inner_temp_hour['data_time'] = pd.to_datetime(inner_temp_hour['data_time'], format="%Y%m%d%H0000")
inner_humidity_hour['data_time'] = pd.to_datetime(inner_humidity_hour['data_time'], format="%Y%m%d%H0000")
inner_PM25_hour['data_time'] = pd.to_datetime(inner_PM25_hour['data_time'], format="%Y%m%d%H0000")
inner_CO2_hour['data_time'] = pd.to_datetime(inner_CO2_hour['data_time'], format="%Y%m%d%H0000")

In [7]:
# inner_temp_hour['data_time'] = inner_temp_hour['data_time'].apply(lambda x: pd.to_datetime(x, format="%Y%m%d%H0000"))
# inner_humidity_hour['data_time'] = inner_humidity_hour['data_time'].apply(lambda x: pd.to_datetime(x, format="%Y%m%d%H0000"))
# inner_PM25_hour['data_time'] = inner_PM25_hour['data_time'].apply(lambda x: pd.to_datetime(x, format="%Y%m%d%H0000"))
# inner_CO2_hour['data_time'] = inner_CO2_hour['data_time'].apply(lambda x: pd.to_datetime(x, format="%Y%m%d%H0000"))


In [8]:
inner_temp_hour['date'] = inner_temp_hour['data_time'].dt.date
inner_temp_hour['hour'] = inner_temp_hour['data_time'].dt.hour

inner_humidity_hour['date'] = inner_humidity_hour['data_time'].dt.date
inner_humidity_hour['hour'] = inner_humidity_hour['data_time'].dt.hour

inner_PM25_hour['date'] = inner_PM25_hour['data_time'].dt.date
inner_PM25_hour['hour'] = inner_PM25_hour['data_time'].dt.hour

inner_CO2_hour['date'] = inner_CO2_hour['data_time'].dt.date
inner_CO2_hour['hour'] = inner_CO2_hour['data_time'].dt.hour

In [9]:
inner_temp_hour.sort_values(by=['date', 'hour'], inplace=True)
inner_humidity_hour.sort_values(by=['date', 'hour'], inplace=True)
inner_PM25_hour.sort_values(by=['date', 'hour'], inplace=True)
inner_CO2_hour.sort_values(by=['date', 'hour'], inplace=True)

In [10]:
inner_temp_hour = inner_temp_hour.pivot(index=['date', 'hour'], columns=['meter'], values=['data_value'])
inner_humidity_hour = inner_humidity_hour.pivot(index=['date', 'hour'], columns=['meter'], values=['data_value'])
inner_PM25_hour = inner_PM25_hour.pivot(index=['date', 'hour'], columns=['meter'], values=['data_value'])
inner_CO2_hour = inner_CO2_hour.pivot(index=['date', 'hour'], columns=['meter'], values=['data_value'])

In [11]:
# inner_temp_hour

In [12]:
cols = []
for col in inner_temp_hour.columns:
    cols.append('inner_temp_' + str(col[1]))
inner_temp_hour.columns = cols

cols = []
for col in inner_humidity_hour.columns:
    cols.append('inner_humidity_' + str(col[1]))
inner_humidity_hour.columns = cols

cols = []
for col in inner_PM25_hour.columns:
    cols.append('inner_PM25_' + str(col[1]))
inner_PM25_hour.columns = cols

cols = []
for col in inner_CO2_hour.columns:
    cols.append('inner_CO2_' + str(col[1]))
inner_CO2_hour.columns = cols

In [13]:
inner_temp_hour.fillna(method='ffill', inplace=True)
inner_humidity_hour.fillna(method='ffill', inplace=True)
inner_PM25_hour.fillna(method='ffill', inplace=True)
inner_PM25_hour.fillna(method='bfill', inplace=True)
inner_CO2_hour.fillna(method='ffill', inplace=True)
inner_CO2_hour.fillna(method='bfill', inplace=True)

In [13]:
# 求室内环境每日的平均数值
inner_temp_daily = inner_temp_hour.reset_index().groupby(by=['date'])[inner_temp_hour.columns].agg('mean')
inner_humidity_daily = inner_humidity_hour.reset_index().groupby(by=['date'])[inner_humidity_hour.columns].agg('mean')
inner_PM25_daily = inner_PM25_hour.reset_index().groupby(by=['date'])[inner_PM25_hour.columns].agg('mean')
inner_CO2_daily = inner_CO2_hour.reset_index().groupby(by=['date'])[inner_CO2_hour.columns].agg('mean')

In [14]:
# inner_temp_hour.isnull().sum(axis= 1).sum(),\
# inner_humidity_hour.isnull().sum(axis= 1).sum(),\
# inner_PM25_hour.isnull().sum(axis= 1).sum(),\
# inner_CO2_hour.isnull().sum(axis= 1).sum()

In [15]:
inner_humidity_hour.shape

(5759, 61)

In [16]:
# pd.to_datetime(1646405999999, unit='ms')

### 2.1 处理每小时天气数据 

In [17]:
weather_hour_cmplt = pd.DataFrame([['500000', '重庆市', '2021-10-21 13:00:00'],
                          ['500000', '重庆市', '2021-10-21 14:00:00'],
                          ['500000', '重庆市', '2021-10-21 15:00:00']],
                          columns=['c_city_code', 'c_city_name', 'c_weather_time'])
# weather_hour_cmplt

weather_hour = weather_hour.append(weather_hour_cmplt)

In [18]:
weather_hour['c_weather_time'] = pd.to_datetime(weather_hour['c_weather_time'])
weather_hour['date'] = weather_hour['c_weather_time'].dt.date
# weather_hour['date'] = pd.to_datetime(weather_hour['date'], format="%Y-%m-%d")
weather_hour['hour'] = weather_hour['c_weather_time'].dt.hour

In [19]:
weather_hour.drop(
[col for col in weather_hour.columns if 'min' in col] +\
[col for col in weather_hour.columns if 'max' in col],
axis=1, inplace=True
)

In [20]:
weather_hour.sort_values(by=['date', 'hour'], inplace=True)
weather_hour.fillna(method='ffill', inplace=True)
weather_hour.set_index(['date', 'hour'], inplace=True)

### 2.2 合并每小时天气和室内环境数据

In [21]:
inner_temp_cols = inner_temp_hour.columns
inner_humidity_cols = inner_humidity_hour.columns

In [22]:
weather_hour.columns

Index(['c_city_code', 'c_city_name', 'c_weather_time', 'c_average_temp',
       'c_average_wind_speed', 'c_average_humidity'],
      dtype='object')

In [23]:
outer_tem_cols = [col for col in weather_hour.columns if 'temp' in col]
outer_humidity_cols = [col for col in weather_hour.columns if 'humidity' in col]

In [24]:
weather_hour = weather_hour.join(inner_temp_hour).join(inner_humidity_hour).join(inner_PM25_hour).join(inner_CO2_hour)


In [25]:
weather_hour.fillna(method='ffill', inplace=True)

In [26]:
# weather_hour.iloc[598:602]

In [27]:
for outer_col in outer_tem_cols:
    for inner_col in inner_temp_cols:
        weather_hour[outer_col + '&' + inner_col + '_diff'] = weather_hour[outer_col] - weather_hour[inner_col]

for outer_col in outer_humidity_cols:
    for inner_col in inner_humidity_cols:
        weather_hour[outer_col + '&' + inner_col + '_diff'] = weather_hour[outer_col] - weather_hour[inner_col]

In [28]:
weather_hour.sort_index(inplace=True)
weather_hour.reset_index(inplace=True)

In [29]:
temp_value_cols = weather_hour.columns.difference(['date', 'hour', 'c_city_code', 'c_city_name', 'c_weather_time'])

In [30]:
weather_hour = weather_hour.pivot(index=['date'], columns=['hour'], values=temp_value_cols)
cols = []
for col in weather_hour.columns:
    cols.append(col[0] + str(col[1]))
weather_hour.columns = cols

In [31]:
# for c in weather_hour.columns:
#     print(c)

### 3.1 处理每日天气数据

In [32]:
weather_daily['c_weather_time'] = pd.to_datetime(weather_daily['c_weather_time'], format="%Y-%m-%d")

In [33]:
weather_daily.drop(
[col for col in weather_daily.columns if 'min' in col] +\
[col for col in weather_daily.columns if 'max' in col],
axis=1, inplace=True
)
weather_daily

,c_city_code,c_city_name,c_weather_time,c_average_temp,c_average_wind_speed,c_average_humidity
0,500000,重庆市,2021-08-01,32.0,6.01,72.0
1,500000,重庆市,2021-08-02,33.5,4.28,69.0
2,500000,重庆市,2021-08-03,33.0,4.90,57.0
3,500000,重庆市,2021-08-04,33.0,3.13,63.0
4,500000,重庆市,2021-08-05,32.5,8.40,72.0
...,...,...,...,...,...,...
238,500000,重庆市,2022-03-27,14.5,3.00,83.0
239,500000,重庆市,2022-03-28,17.0,8.40,73.0
240,500000,重庆市,2022-03-29,17.5,8.40,66.0
241,500000,重庆市,2022-03-30,16.0,3.00,91.0


In [34]:
weather_daily.sort_values(by=['c_weather_time'], inplace=True)
weather_daily.set_index(['c_weather_time'], inplace=True)

### 3.2 合并每日天气和室内环境数据

In [35]:
inner_temp_cols = inner_temp_daily.columns
inner_humidity_cols = inner_humidity_daily.columns

In [36]:
weather_daily.columns
# inner_temp_daily.columns

Index(['c_city_code', 'c_city_name', 'c_average_temp', 'c_average_wind_speed',
       'c_average_humidity'],
      dtype='object')

In [37]:
outer_tem_cols = [col for col in weather_daily.columns if 'temp' in col]
outer_humidity_cols = [col for col in weather_daily.columns if 'humidity' in col]

In [38]:
weather_daily = weather_daily.join(inner_temp_daily).join(inner_humidity_daily).join(inner_PM25_daily).join(inner_CO2_daily)

In [39]:
for outer_col in outer_tem_cols:
    for inner_col in inner_temp_cols:
        weather_daily[outer_col + '&' + inner_col + '_diff'] = weather_daily[outer_col] - weather_daily[inner_col]

for outer_col in outer_humidity_cols:
    for inner_col in inner_humidity_cols:
        weather_daily[outer_col + '&' + inner_col + '_diff'] = weather_daily[outer_col] - weather_daily[inner_col]

In [40]:
weather_daily

,c_city_code,c_city_name,c_average_temp,c_average_wind_speed,c_average_humidity,inner_temp_4001,inner_temp_4005,inner_temp_4007,inner_temp_4008,inner_temp_4009,...,c_average_humidity&inner_humidity_20037_diff,c_average_humidity&inner_humidity_20039_diff,c_average_humidity&inner_humidity_20040_diff,c_average_humidity&inner_humidity_20041_diff,c_average_humidity&inner_humidity_20042_diff,c_average_humidity&inner_humidity_20043_diff,c_average_humidity&inner_humidity_20045_diff,c_average_humidity&inner_humidity_20046_diff,c_average_humidity&inner_humidity_20047_diff,c_average_humidity&inner_humidity_20061_diff
c_weather_time,,,,,,,,,,,,,,,,,,,,,
2021-08-01,500000,重庆市,32.0,6.01,72.0,32.904167,30.250000,27.079167,26.608333,26.104167,...,8.158333,7.183333,12.950000,10.908333,7.595833,7.850000,10.533333,8.345833,6.637500,10.345833
2021-08-02,500000,重庆市,33.5,4.28,69.0,32.841667,30.441667,27.112500,26.920833,26.187500,...,0.787500,-0.579167,4.316667,4.054167,0.279167,1.691667,2.433333,0.450000,-1.258333,2.741667
2021-08-03,500000,重庆市,33.0,4.90,57.0,32.950000,30.483333,27.162500,27.070833,26.337500,...,-9.066667,-9.983333,-6.087500,-6.379167,-9.691667,-8.666667,-7.379167,-7.570833,-10.058333,-8.595833
2021-08-04,500000,重庆市,33.0,3.13,63.0,33.029167,30.520833,27.200000,26.529167,26.391667,...,-3.129167,-4.120833,-0.187500,-0.187500,-3.504167,-2.533333,0.062500,-1.412500,-4.287500,-3.125000
2021-08-05,500000,重庆市,32.5,8.40,72.0,33.020833,30.583333,27.316667,27.079167,26.270833,...,3.845833,1.958333,6.737500,7.287500,3.983333,4.333333,6.754167,4.950000,2.091667,3.412500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-27,500000,重庆市,14.5,3.00,83.0,22.683333,22.495833,23.462500,20.983333,18.033333,...,16.212500,17.437500,11.758333,13.429167,18.379167,23.958333,18.070833,15.950000,19.100000,16.462500
2022-03-28,500000,重庆市,17.0,8.40,73.0,22.654167,22.416667,23.483333,21.625000,19.220833,...,17.120833,14.850000,13.137500,14.354167,16.850000,20.383333,11.308333,17.654167,18.616667,14.441667
2022-03-29,500000,重庆市,17.5,8.40,66.0,22.895833,22.462500,23.820833,22.233333,20.258333,...,12.450000,9.791667,10.512500,11.654167,12.708333,8.104167,7.512500,13.800000,13.770833,8.833333


### 4. 合并每日和每时天气数据

In [41]:
# weather_hour.drop(['c_city_code', 'c_city_name'], axis=1, inplace=True)
weather_daily.drop(['c_city_code', 'c_city_name'], axis=1, inplace=True)
# df_weather_dt.drop(['c_city_code', 'c_city_name'], axis=1, inplace=True)

In [42]:
# weather_daily.set_index('c_weather_time', inplace=True)
# 合并两个天气数据集
df_weather_dt = weather_daily.join(weather_hour)

In [43]:
weather_hour = weather_hour.T
weather_daily = weather_daily.T
df_weather_dt = df_weather_dt.T

In [44]:
weather_hour

date,2021-08-01,2021-08-02,2021-08-03,2021-08-04,2021-08-05,2021-08-06,2021-08-07,2021-08-08,2021-08-09,2021-08-10,...,2022-03-22,2022-03-23,2022-03-24,2022-03-25,2022-03-26,2022-03-27,2022-03-28,2022-03-29,2022-03-30,2022-03-31
c_average_humidity0,49.0,56.0,53.0,48.0,54.0,64.0,59.0,82.0,94.0,80.0,...,92.0,87.0,81.0,81.0,85.0,95.0,95.0,66.0,66.0,95.0
c_average_humidity1,48.0,69.0,58.0,51.0,56.0,66.0,62.0,83.0,94.0,82.0,...,95.0,87.0,80.0,81.0,90.0,95.0,95.0,71.0,65.0,96.0
c_average_humidity2,51.0,61.0,60.0,54.0,59.0,62.0,65.0,84.0,94.0,83.0,...,96.0,87.0,83.0,80.0,93.0,96.0,95.0,68.0,70.0,96.0
c_average_humidity3,60.0,64.0,62.0,56.0,60.0,59.0,65.0,84.0,90.0,85.0,...,95.0,90.0,84.0,79.0,93.0,96.0,96.0,72.0,73.0,97.0
c_average_humidity4,59.0,67.0,63.0,57.0,62.0,59.0,67.0,87.0,89.0,89.0,...,96.0,91.0,87.0,79.0,95.0,97.0,97.0,74.0,74.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
inner_temp_412019,26.4,27.3,26.3,26.9,26.4,27.6,26.6,25.8,26.3,25.4,...,23.2,23.3,23.4,23.3,22.8,22.7,23.2,23.1,22.8,22.2
inner_temp_412020,26.2,27.7,26.3,26.9,26.2,27.8,26.6,25.7,26.3,25.4,...,23.3,23.4,23.4,23.5,22.8,22.6,23.2,23.1,22.9,22.2
inner_temp_412021,26.1,27.6,26.6,25.9,26.9,27.9,26.5,25.7,26.4,25.1,...,23.2,23.3,23.4,23.5,22.6,22.4,23.2,23.1,22.9,22.2
inner_temp_412022,26.4,27.4,26.7,26.0,26.6,27.9,26.3,25.6,26.5,24.5,...,23.2,23.2,23.3,23.4,22.5,22.3,23.1,23.0,22.8,22.1


In [45]:
# 导出数据
weather_hour.to_pickle('./work/AI_data/weather_hour.pkl')
weather_daily.to_pickle('./work/AI_data/weather_daily.pkl')
df_weather_dt.to_pickle('./work/AI_data/df_weather_dt.pkl')